In [1]:
from services.api.pipelines.import_functions import MonarkImport, monark_import
from services.api.pipelines.data_parsing_functions import parse_budget_data, parse_transaction_data
import pandas as pd
from datetime import datetime
import json
from datetime import datetime, timedelta

data_parsing_functions.py loaded successfully
parse_budget_data function available: True


In [2]:
import gql
gql.__version__

'4.0.0'

In [3]:
from config import Settings

In [4]:
Settings.MONARK_DD_ID

'f663a6dc-ab58-4fa2-9dfd-06baa39eaf9d'

In [5]:
imports = await monark_import("Mg182179!", "mariogj1987@gmail.com")



Using saved session found at .mm/mm_session.pickle
Logged in to MonarchMoney successfully.
📅 Fetching transactions from 2025-08-01 to 2025-09-22


In [6]:
budget_data = parse_budget_data(imports['budget'])
print(json.dumps(json.loads(budget_data), indent = 4, sort_keys=True, ensure_ascii=False))

[
    {
        "actual_amount": 3675.0,
        "category_budget_variability": "fixed",
        "category_group_name": "Income",
        "category_group_type": "income",
        "category_name": "Paychecks",
        "month": "2025-09-01",
        "planned_cash_flow_amount": 10000.0,
        "remaining_amount": 6325.0,
        "remaining_amount_percent": 0.6325
    },
    {
        "actual_amount": 20.0,
        "category_budget_variability": "flexible",
        "category_group_name": "Income",
        "category_group_type": "income",
        "category_name": "Savings",
        "month": "2025-09-01",
        "planned_cash_flow_amount": 6000.0,
        "remaining_amount": 5980.0,
        "remaining_amount_percent": 0.9967
    },
    {
        "actual_amount": 42.0,
        "category_budget_variability": "flexible",
        "category_group_name": "Income",
        "category_group_type": "income",
        "category_name": "Dividends & Capital Gains",
        "month": "2025-09-01",
       

In [8]:
imports['transactions']

{'allTransactions': {'totalCount': 352,
  'results': [{'id': '222678562304322212',
    'amount': 15175.68,
    'pending': False,
    'date': '2025-09-22',
    'hideFromReports': False,
    'plaidName': 'REDEMPTION FROM CORE ACCOUNT FIDELITY GOVERNMENT MONEY MARKET (SPAXX) MORNING TRADE (Cash)',
    'notes': None,
    'isRecurring': False,
    'reviewStatus': 'reviewed',
    'needsReview': False,
    'attachments': [],
    'isSplitTransaction': False,
    'createdAt': '2025-09-23T21:40:25.749313+00:00',
    'updatedAt': '2025-09-23T21:40:25.749352+00:00',
    'category': {'id': '221275213886548045',
     'name': 'Buy',
     '__typename': 'Category'},
    'merchant': {'name': 'Redemption From Core Account Fidelity Government Money Market (Spaxx) Morning Trade (Cash)',
     'id': '222678562092510042',
     'transactionsCount': 2,
     '__typename': 'Merchant'},
    'account': {'id': '220868046903732310',
     'displayName': 'Rollover IRA (...*****4160)',
     '__typename': 'Account'},
   

In [7]:
txn_data = parse_transaction_data(imports['transactions'])
print(json.dumps(json.loads(txn_data), indent = 4, sort_keys=True, ensure_ascii=False))

[
    {
        "account_name": "CHASE SAVINGS (...6614)",
        "amount": 900.0,
        "category_id": "138409570308352146",
        "category_name": "Unemployment",
        "createdAt": "2025-09-23T19:31:35.560539+00:00",
        "description": "Money Network UI Deposit PPD ID: 364483541",
        "merchant_id": "222579909494467140",
        "merchant_name": "Certificate of Origin Money Network",
        "transaction_id": "222670456678673400",
        "updatedAt": "2025-09-23T19:31:35.560550+00:00"
    },
    {
        "account_name": "CREDIT CARD (...1160)",
        "amount": -160.68,
        "category_id": "138409570351343792",
        "category_name": "Child Activities",
        "createdAt": "2025-09-23T19:31:23.141945+00:00",
        "description": "SCOUTING* REGISTRATION",
        "merchant_id": "222670442632997824",
        "merchant_name": "Scouting Registration",
        "transaction_id": "222670443573570167",
        "updatedAt": "2025-09-23T19:31:23.141949+00:00"
    },


In [10]:
import json

def print_structure(obj, max_depth=2):
    """Print the structure of the object in a readable format"""
    
    def truncate_structure(obj, depth=0):
        if depth >= max_depth:
            return f"<{type(obj).__name__}>"
        
        if isinstance(obj, dict):
            return {k: truncate_structure(v, depth + 1) for k, v in list(obj.items())[:5]}
        elif isinstance(obj, list):
            if len(obj) > 0:
                return [truncate_structure(obj[0], depth + 1)]
            return []
        else:
            return f"<{type(obj).__name__}>"
    
    structure = truncate_structure(obj)
    print(json.dumps(structure, indent=2, default=str))

print_structure(imports['budget'])

{
  "budgetSystem": "<str>",
  "budgetData": {
    "monthlyAmountsByCategory": "<list>",
    "monthlyAmountsByCategoryGroup": "<list>",
    "monthlyAmountsForFlexExpense": "<dict>",
    "totalsByMonth": "<list>",
    "__typename": "<str>"
  },
  "categoryGroups": [
    "<dict>"
  ],
  "goalsV2": []
}


In [14]:
imports['budget']['categoryGroups']

#categoryGroups has the category name an all the subgroups (categories) within it 
# within in category there is budgetVariability which is the flexible vs fixed vs non monthly view I like, we would need to re arrange categories to summ up to this instead of category group
# type , which is a category group level data point tell syou if the category group is income or expense
# excludeFromBudget is a category level data point that tells you if the category is excluded from budget calculations

# category group: 'id': '138409570268506252', name : Income
# category : 'id': '138409570304157839', name: 'Paychecks'
\
# Parsing data actions for imports['budget']['categoryGroups']: 
# 1. Create a dataframe with category group, id, category, id, type, budget variabilility and excludeFromBudget from imports['budget']['categoryGroups']
# 2. remove all rows where excludeFromBudget is True
# 3 remove all types that are not income or expense





[{'id': '138409570268506252',
  'name': 'Income',
  'order': 0,
  'type': 'income',
  'budgetVariability': 'flexible',
  'updatedAt': '2025-09-17T04:16:36.560955+00:00',
  'groupLevelBudgetingEnabled': False,
  'categories': [{'id': '138409570304157839',
    'name': 'Paychecks',
    'icon': '💵',
    'order': 0,
    'budgetVariability': 'fixed',
    'excludeFromBudget': False,
    'isSystemCategory': True,
    'updatedAt': '2023-03-08T18:00:52.456408+00:00',
    'group': {'id': '138409570268506252',
     'type': 'income',
     'budgetVariability': 'flexible',
     'groupLevelBudgetingEnabled': False,
     '__typename': 'CategoryGroup'},
    'rolloverPeriod': None,
    '__typename': 'Category'},
   {'id': '138409570305206416',
    'name': 'Interest',
    'icon': '💸',
    'order': 1,
    'budgetVariability': 'flexible',
    'excludeFromBudget': True,
    'isSystemCategory': True,
    'updatedAt': '2025-05-28T03:09:41.567998+00:00',
    'group': {'id': '138409570268506252',
     'type': 'i

In [15]:
imports['budget']['budgetData']['monthlyAmountsByCategory']

# monthlyAmountsByCategory has the budgeted amounts by category id and month
# plannedCashFlowAmount, within Monthly Amount is the buggeted amount, actualAmount is the actual spent amount, remainingAmount is the difference between the two , which is the amount left to spend until the end of the month
# Monthly Amount is a dictionary with monthly data, we need to take only the current month data. 


# Parsing data actions for imports['budget']['budgetData']['monthlyAmountsByCategory']:

# 1. Create a dataframe with category id, month, plannedCashFlowAmount, actualAmount and remainingAmount from imports['budget']['budgetData']['monthlyAmountsByCategory']
# 2. filter the dataframe to only include the current month
# 3. left join with the category dataframe on category id to get category name, category group name, type and budget variability

[{'category': {'id': '138409570373363903', '__typename': 'Category'},
  'monthlyAmounts': [{'month': '2025-08-01',
    'plannedCashFlowAmount': 0.0,
    'plannedSetAsideAmount': 0.0,
    'actualAmount': 0.0,
    'remainingAmount': 0.0,
    'previousMonthRolloverAmount': None,
    'rolloverType': None,
    'cumulativeActualAmount': 0.0,
    'rolloverTargetAmount': None,
    '__typename': 'BudgetMonthlyAmounts'},
   {'month': '2025-09-01',
    'plannedCashFlowAmount': 0.0,
    'plannedSetAsideAmount': 0.0,
    'actualAmount': 0.0,
    'remainingAmount': 0.0,
    'previousMonthRolloverAmount': None,
    'rolloverType': None,
    'cumulativeActualAmount': 0.0,
    'rolloverTargetAmount': None,
    '__typename': 'BudgetMonthlyAmounts'},
   {'month': '2025-10-01',
    'plannedCashFlowAmount': 0.0,
    'plannedSetAsideAmount': 0.0,
    'actualAmount': 0.0,
    'remainingAmount': 0.0,
    'previousMonthRolloverAmount': None,
    'rolloverType': None,
    'cumulativeActualAmount': 0.0,
    'rol

In [ ]:

def parse_budget_data(budget_data):
    """
    Parse budget data into structured DataFrames
    """
    
    # Part 1: Parse categoryGroups into categories DataFrame

    categories_data = []
    
    # 1. Create a dataframe with category group and category 

    for category_group in budget_data['categoryGroups']:
        group_id = category_group['id']
        group_name = category_group['name']
        group_type = category_group.get('type', None)  # income/expense
        group_budget_variability = category_group.get('budgetVariability', None)
        
        # Process each category within the group
        for category in category_group.get('categories', []):
            categories_data.append({
                'category_group_id': group_id,
                'category_group_name': group_name,
                'category_group_type': group_type,
                'category_group_budget_variability': group_budget_variability,
                'category_id': category['id'],
                'category_name': category['name'],
                'category_budget_variability': category.get('budgetVariability', None),
                'exclude_from_budget': category.get('excludeFromBudget', False)
            })
    
    categories_df = pd.DataFrame(categories_data)
    
    # 2. Remove rows where excludeFromBudget is True
    categories_df = categories_df[categories_df['exclude_from_budget'] == False]
    
    # 3. Remove types that are not income or expense
    categories_df = categories_df[categories_df['category_group_type'].isin(['income', 'expense'])]
    
    
    # Part 2: Parse monthlyAmountsByCategory into budget amounts DataFrame
        
    budget_amounts_data = []
    current_month = datetime.now().strftime('%Y-%m-01')  # Format as YYYY-MM-01 to match data
    
    # 1. Create a dataframe with category id, month, plannedCashFlowAmount, actualAmount and remainingAmount

    for category_budget in budget_data['budgetData']['monthlyAmountsByCategory']:

        category_id = category_budget['category']['id']
        
        # Process each month's data
        for monthly_amount in category_budget.get('monthlyAmounts', []):
            month = monthly_amount.get('month')
            
            budget_amounts_data.append({
                'category_id': category_id,
                'month': month,
                'planned_cash_flow_amount': monthly_amount.get('plannedCashFlowAmount', 0),
                'actual_amount': monthly_amount.get('actualAmount', 0),
                'remaining_amount': monthly_amount.get('remainingAmount', 0)
            })
    
    
    budget_amounts_df = pd.DataFrame(budget_amounts_data)
    
    
    # 2. Filter to current month only
    current_month_df = budget_amounts_df[budget_amounts_df['month'] == current_month]
    
   
    # 3. Left join budget amounts with categories on category_id
    complete_budget_df= categories_df.merge(
        current_month_df, 
        on='category_id', 
        how='left'
    )
    
    # 4. remove categories where both bugeted and actual amounts are zero or NaN
    complete_budget_df = complete_budget_df[~((complete_budget_df['planned_cash_flow_amount'].fillna(0) == 0) & (complete_budget_df['actual_amount'].fillna(0) == 0))]

        # Select only the required columns
    columns_to_keep = [
        'category_group_type', 
        'category_budget_variability', 
        'category_group_name', 
        'category_name',
        'month', 
        'planned_cash_flow_amount', 
        'actual_amount', 
        'remaining_amount'
    ]
    
    # Filter columns
    simplified_df = complete_budget_df[columns_to_keep].copy()
    
   
    simplified_df['sort_type'] = simplified_df['category_group_type'].map({
        'income': 0, 
        'expense': 1
    })
    
    # Sort by type first (income=0, expense=1), then by budget variability
    simplified_df = simplified_df.sort_values([
        'sort_type', 
        'category_budget_variability'
    ]).reset_index(drop=True)
    
    # Remove the temporary sorting column
    simplified_df = simplified_df.drop('sort_type', axis=1)

    
    return simplified_df


final_budget_df = parse_budget_data(imports['budget'])

summary_by_type = final_budget_df.groupby('category_group_type')[['planned_cash_flow_amount', 'actual_amount', 'remaining_amount']].sum()


In [35]:
final_budget_df.head(10)

,category_group_type,category_budget_variability,category_group_name,category_name,month,planned_cash_flow_amount,actual_amount,remaining_amount
0,income,fixed,Income,Paychecks,2025-09-01,10000.0,3675.0,6325.0
1,income,flexible,Income,Savings,2025-09-01,6000.0,20.0,5980.0
2,income,None,Income,Dividends & Capital Gains,2025-09-01,0.0,42.0,-42.0
3,expense,fixed,Housing,Mortgage,2025-09-01,6680.0,6683.0,-3.0
4,expense,fixed,Bills & Utilities,Garbage,2025-09-01,180.0,277.0,-97.0
5,expense,fixed,Bills & Utilities,Water,2025-09-01,300.0,200.0,100.0
6,expense,fixed,Bills & Utilities,Gas & Electric,2025-09-01,400.0,0.0,400.0
7,expense,fixed,Bills & Utilities,Internet & Cable,2025-09-01,170.0,269.0,-99.0
8,expense,fixed,Bills & Utilities,Phone,2025-09-01,100.0,0.0,100.0
9,expense,fixed,Bills & Utilities,Postage & Shipping,2025-09-01,10.0,0.0,10.0


In [ ]:
def create_simplified_budget_df(final_budget_df):
    """
    Create a simplified budget dataframe with specific columns and sorting
    
    Keeps only: category_group_type, category_budget_variability, category_group_name, category_name
               month, planned_cash_flow_amount, actual_amount, remaining_amount
    
    Sorts by: income first, then by budget variability
    """
    


    
    return simplified_df

# Create the simplified dataframe
simplified_budget_df = create_simplified_budget_df(final_budget_df)


In [33]:
simplified_budget_df

,category_group_type,category_budget_variability,category_group_name,category_name,month,planned_cash_flow_amount,actual_amount,remaining_amount
0,income,fixed,Income,Paychecks,2025-09-01,10000.0,3675.0,6325.0
1,income,flexible,Income,Savings,2025-09-01,6000.0,20.0,5980.0
2,income,None,Income,Dividends & Capital Gains,2025-09-01,0.0,42.0,-42.0
3,expense,fixed,Housing,Mortgage,2025-09-01,6680.0,6683.0,-3.0
4,expense,fixed,Bills & Utilities,Garbage,2025-09-01,180.0,277.0,-97.0
5,expense,fixed,Bills & Utilities,Water,2025-09-01,300.0,200.0,100.0
6,expense,fixed,Bills & Utilities,Gas & Electric,2025-09-01,400.0,0.0,400.0
7,expense,fixed,Bills & Utilities,Internet & Cable,2025-09-01,170.0,269.0,-99.0
8,expense,fixed,Bills & Utilities,Phone,2025-09-01,100.0,0.0,100.0
9,expense,fixed,Bills & Utilities,Postage & Shipping,2025-09-01,10.0,0.0,10.0


In [ ]:
def parse_transaction_data(transactions_data):
    """
    Parse transaction data into a structured DataFrame
    
    Creates DataFrame with columns:
    - transaction_id (from id)
    - amount
    - description (plaidName)
    - category_id 
    - category_name
    - merchant_id
    - merchant_name
    - createdAt
    - updatedAt
    
    Filters to keep only transactions from current and previous day
    """
    
    # Get the results from the nested structure
    transaction_results = transactions_data.get('allTransactions', {}).get('results', [])
    
    print(f"📊 Processing {len(transaction_results)} transactions...")
    
    transactions_list = []
    
    for transaction in transaction_results:
        # Extract basic transaction info
        transaction_id = transaction.get('id')
        amount = transaction.get('amount', 0)
        description = transaction.get('plaidName', '')
        
        # Extract timestamps
        created_at = transaction.get('createdAt')
        updated_at = transaction.get('updatedAt')
        
        # Extract category info (safely handle missing category)
        category = transaction.get('category', {})
        category_id = category.get('id', None) if category else None
        category_name = category.get('name', None) if category else None
        
        # Extract merchant info (safely handle missing merchant)
        merchant = transaction.get('merchant', {})
        merchant_id = merchant.get('id', None) if merchant else None
        merchant_name = merchant.get('name', None) if merchant else None
        
        # Add to list
        transactions_list.append({
            'transaction_id': transaction_id,
            'amount': amount,
            'description': description,
            'category_id': category_id,
            'category_name': category_name,
            'merchant_id': merchant_id,
            'merchant_name': merchant_name,
            'createdAt': created_at,
            'updatedAt': updated_at
        })
    
    # Create DataFrame
    transactions_df = pd.DataFrame(transactions_list)
    

    # Convert timestamp columns to datetime
    transactions_df['createdAt'] = pd.to_datetime(transactions_df['createdAt'], errors='coerce')
    transactions_df['updatedAt'] = pd.to_datetime(transactions_df['updatedAt'], errors='coerce')
    
    # Create a column for the most recent timestamp
    transactions_df['most_recent_timestamp'] = transactions_df[['createdAt', 'updatedAt']].max(axis=1)
    
    # Calculate date range (current and previous day)
    now = datetime.now()
    yesterday = now - timedelta(days=1)
    
    # Filter transactions from current and previous day
    recent_transactions = transactions_df[
        transactions_df['most_recent_timestamp'].dt.date >= yesterday.date()
    ].copy()
    
    print(f"🔍 Filtered to {len(recent_transactions)} transactions from {yesterday.date()} onwards")
    
    # Drop the temporary column
    recent_transactions = recent_transactions.drop('most_recent_timestamp', axis=1)
    
    # Convert timestamps to human-readable format before JSON conversion
    recent_transactions['createdAt'] = recent_transactions['createdAt'].dt.strftime('%Y-%m-%d %H:%M:%S')
    recent_transactions['updatedAt'] = recent_transactions['updatedAt'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    transactions_json = recent_transactions.to_json(orient='records')
    
    return transactions_json

# Parse the transaction data
transactions_json = parse_transaction_data(imports['transactions'])

print(json.dumps(json.loads(transactions_json) , indent=4, sort_keys=True, ensure_ascii=False))

📊 Processing 100 transactions...
📋 Created DataFrame with 100 total transactions
🔍 Filtered to 3 transactions from 2025-09-21 onwards
[
    {
        "amount": 66.81,
        "category_id": "221275215166859342",
        "category_name": "Dividends & Capital Gains",
        "createdAt": "2025-09-22 11:42:47",
        "description": "VANGUARD DEVELOPED MARKETS INDEX ADMIRAL",
        "merchant_id": "222550364859034618",
        "merchant_name": "Vanguard Developed Markets Index Admiral",
        "transaction_id": "222550365494471235",
        "updatedAt": "2025-09-22 11:42:47"
    },
    {
        "amount": -66.81,
        "category_id": "221275213886548045",
        "category_name": "Buy",
        "createdAt": "2025-09-22 11:42:47",
        "description": "VANGUARD DEVELOPED MARKETS INDEX ADMIRAL",
        "merchant_id": "222550364859034618",
        "merchant_name": "Vanguard Developed Markets Index Admiral",
        "transaction_id": "222550365494471234",
        "updatedAt": "2025-09